In [2]:
import json 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time 
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.common import exceptions  
import re
import pandas as pd
import numpy as np


In [3]:
import mysql.connector

conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="FeB1st@2k",
  database="genmedi"
)


In [ ]:
def checkTableExists(dbcon, tablename):
    dbcur = dbcon.cursor()
    dbcur.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(tablename.replace('\'', '\'\'')))
    if dbcur.fetchone()[0] == 1:
        dbcur.close()
        return True

    dbcur.close()
    return False

In [4]:
cursor = conn.cursor()
if checkTableExists(conn,'composition_'):
    cursor = conn.cursor()
    cursor.execute("""create table composition_(Name_of_the_salt varchar(200),Link varchar(200),ID varchar(200))""")
if checkTableExists(conn,'medicine_list_'):
    cursor=conn.cursor()
    cursor.execute("""create table medicine_list_(medicine_name varchar(200),mfg_name varchar(200),comp_id varchar(200))""")    

ProgrammingError: 1050 (42S01): Table 'composition_' already exists

In [6]:
cursor.execute("select * from composition_")

for x in cursor:
    print(x)

('Aceclofenac 100MG + Paracetamol 325MG Tablets', 'Aceclofenac-100-MG-and-Paracetamol-325-MG', '5370')
('Aceclofenac 100 MG Tablets', 'Aceclofenac-100-MG', '3089')
('Diclofenac 50 MG+ paracetamol 325 MG+ chlorzoxazone 500 MG tablets', 'Paracetamol-325-MG-and-Chlorzoxazone-500-MG', '21675')
('Diclofenac Sodium 50MG + Serratiopeptidase 10 MG Tab', 'Diclofenac-50-MG-and-Serratiopeptidase-10-MG', '9533')
('Etoricoxilb 120MG Tab ', 'Etoricoxib-120-MG', '5524')
('Etoricoxilb 90MG Tab', 'Etoricoxib-90-MG', '5523')
('Ibuprofen film coated  Tablets 200MG', 'Ibuprofen-200-MG', '3526')
('Ibuprofen Tablets   400MG', 'Ibuprofen-400-MG', '5799')
('Nimesulide 100 MG Tab', 'Nimesulide-100-MG', '3269')
('Paracetamol 325MG+ Diclofenac Sodium 50 MG Tab', 'Diclofenac-50-MG-and-Paracetamol-325-MG', '6469')
('Serratiopeptidase Tablets 10 MG', 'Serratiopeptidase-10-MG', '5753')
('Allopurinol 100 MG Tabs', 'Allopurinol-100-MG', '5253')
('Tranexamic Acid 500 mg+ Mefenamic Acid 250 mg Tablets', 'Tranexamic-acid

In [7]:
import pandas as pd
info=pd.read_excel('Drugs_info.xlsx')
print(info.head())

                                    Name_of_the_salt  \
0      Aceclofenac 100MG + Paracetamol 325MG Tablets   
1                         Aceclofenac 100 MG Tablets   
2  Diclofenac 50 MG+ paracetamol 325 MG+ chlorzox...   
3  Diclofenac Gel ( Diclofenec Diethylamine 1.16 ...   
4  Diclofenac Sodium 50MG + Serratiopeptidase 10 ...   

                                         Search Link  Link  ID  
0  https://www.medplusmart.com/compositionProduct...   NaN NaN  
1  https://www.medplusmart.com/compositionProduct...   NaN NaN  
2  https://www.medplusmart.com/compositionProduct...   NaN NaN  
3                                                NaN   NaN NaN  
4  https://www.medplusmart.com/compositionProduct...   NaN NaN  


In [8]:
x=[]
y=[]
for i in range(len(info)):
    if pd.isnull(info.loc[i,'Search Link'])==False:
        x.append((info.loc[i,'Search Link']).split('/')[-2])
        y.append((info.loc[i,'Search Link']).split('/')[-1])
    else:
        x.append('Sorry')
        y.append(0)
print(x)

['Aceclofenac-100-MG-and-Paracetamol-325-MG', 'Aceclofenac-100-MG', 'Paracetamol-325-MG-and-Chlorzoxazone-500-MG', 'Sorry', 'Diclofenac-50-MG-and-Serratiopeptidase-10-MG', 'Sorry', 'Sorry', 'Etoricoxib-120-MG', 'Etoricoxib-90-MG', 'Ibuprofen-200-MG', 'Ibuprofen-400-MG', 'Nimesulide-100-MG', 'Diclofenac-50-MG-and-Paracetamol-325-MG', 'Serratiopeptidase-10-MG', 'Sorry', 'Sorry', 'Allopurinol-100-MG', 'Tranexamic-acid-500-MG-and-Mefenamic-Acid-250-MG', 'Paracetamol-325-MG-and-Tramadol-37-5-MG', 'Paracetamol-650-MG', 'Sorry', 'Piroxicam-20-MG', 'Mefenamic-Acid-100-MG-5ML', 'Aceclofenac-200-MG', 'Thiocolchicoside-4-MG-and-Aceclofenac-100-MG', 'Baclofen-10-MG', 'Sorry', 'Mefenamic-Acid-500-MG-and-Paracetamol-325-MG', 'Tramadol-100-MG', 'Paracetamol-325-MG-and-Diclofenac-50-MG', 'Sorry', 'Nimesulide-1-percentW-W', 'Mefenamic-Acid-250-MG-and-Dicyclomine-10-MG', 'Paracetamol-125-MG', 'Diacerein-50-MG-and-Glucosamine-750-MG', 'Sorry', 'Piroxicam-20-MG', 'Piroxicam-20-MG-ML', 'Piroxicam-40-MG-INJ

In [9]:
info['Link']=x
info['ID']=y
print(info.head())

                                    Name_of_the_salt  \
0      Aceclofenac 100MG + Paracetamol 325MG Tablets   
1                         Aceclofenac 100 MG Tablets   
2  Diclofenac 50 MG+ paracetamol 325 MG+ chlorzox...   
3  Diclofenac Gel ( Diclofenec Diethylamine 1.16 ...   
4  Diclofenac Sodium 50MG + Serratiopeptidase 10 ...   

                                         Search Link  \
0  https://www.medplusmart.com/compositionProduct...   
1  https://www.medplusmart.com/compositionProduct...   
2  https://www.medplusmart.com/compositionProduct...   
3                                                NaN   
4  https://www.medplusmart.com/compositionProduct...   

                                           Link     ID  
0     Aceclofenac-100-MG-and-Paracetamol-325-MG   5370  
1                            Aceclofenac-100-MG   3089  
2   Paracetamol-325-MG-and-Chlorzoxazone-500-MG  21675  
3                                         Sorry      0  
4  Diclofenac-50-MG-and-Serratiopeptidase

In [10]:
info.isnull().sum()

Name_of_the_salt     0
Search Link         17
Link                 0
ID                   0
dtype: int64

In [11]:
info.dtypes

Name_of_the_salt    object
Search Link         object
Link                object
ID                  object
dtype: object

In [12]:
info.dropna(inplace=True)

In [13]:
len(info)

198

In [14]:
info.drop(['Search Link'],axis=1,inplace=True)

In [13]:
# creating column list for insertion
cols = "`,`".join([str(i) for i in info.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in info.iterrows():
    sql = "INSERT INTO `composition_` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit()

In [14]:
cursor.execute("""select * from composition_""")
for x in cursor:
    print(x)

('Aceclofenac 100MG + Paracetamol 325MG Tablets', 'Aceclofenac-100-MG-and-Paracetamol-325-MG', '5370')
('Aceclofenac 100 MG Tablets', 'Aceclofenac-100-MG', '3089')
('Diclofenac 50 MG+ paracetamol 325 MG+ chlorzoxazone 500 MG tablets', 'Paracetamol-325-MG-and-Chlorzoxazone-500-MG', '21675')
('Diclofenac Sodium 50MG + Serratiopeptidase 10 MG Tab', 'Diclofenac-50-MG-and-Serratiopeptidase-10-MG', '9533')
('Etoricoxilb 120MG Tab ', 'Etoricoxib-120-MG', '5524')
('Etoricoxilb 90MG Tab', 'Etoricoxib-90-MG', '5523')
('Ibuprofen film coated  Tablets 200MG', 'Ibuprofen-200-MG', '3526')
('Ibuprofen Tablets   400MG', 'Ibuprofen-400-MG', '5799')
('Nimesulide 100 MG Tab', 'Nimesulide-100-MG', '3269')
('Paracetamol 325MG+ Diclofenac Sodium 50 MG Tab', 'Diclofenac-50-MG-and-Paracetamol-325-MG', '6469')
('Serratiopeptidase Tablets 10 MG', 'Serratiopeptidase-10-MG', '5753')
('Allopurinol 100 MG Tabs', 'Allopurinol-100-MG', '5253')
('Tranexamic Acid 500 mg+ Mefenamic Acid 250 mg Tablets', 'Tranexamic-acid

In [15]:
info

,Name_of_the_salt,Link,ID
0,Aceclofenac 100MG + Paracetamol 325MG Tablets,Aceclofenac-100-MG-and-Paracetamol-325-MG,5370
1,Aceclofenac 100 MG Tablets,Aceclofenac-100-MG,3089
2,Diclofenac 50 MG+ paracetamol 325 MG+ chlorzox...,Paracetamol-325-MG-and-Chlorzoxazone-500-MG,21675
4,Diclofenac Sodium 50MG + Serratiopeptidase 10 ...,Diclofenac-50-MG-and-Serratiopeptidase-10-MG,9533
7,Etoricoxilb 120MG Tab,Etoricoxib-120-MG,5524
...,...,...,...
210,Escitalopram 10 mg With Clonazepam 0.5 mg,Escitalopram-10-MG-and-Clonazepam-0-5-MG,7891
211,Nortrtyline Tablet 25 mg Tablet,Nortriptyline-25-MG,4951
212,Etizolam Tablet 0.5 mg,Etizolam-0-5-MG,8516
213,Adenosine Injection 3mg / ml,Adenosine-3-MG-ML,24395


In [16]:
def scrape_it(df):
    product_details=pd.DataFrame()
    medicine_name=[]
    mfg_name=[]
    ids=[]
    lin=df['Link']
    id_=df['ID']
    browser = None
    try:
        browser = webdriver.Chrome(executable_path='C:\\Users\\PRASANNA\\chromedriver_win32\\chromedriver.exe')
    except Exception as error:
        print(error)
    for links,_id in zip(lin,id_):
        link= 'https://www.medplusmart.com/compositionProducts/'+links+'/'+_id
        name=links
        try:
                    browser.get(link)
        except Exception as err:
                    print(str(err))
        else:
                    print('\nSuccessfully Accessed:',link)


        medicine_name_per_composition = browser.find_elements_by_xpath('//*[@id="productSearch"]/tbody/tr/td[1]/div/span/a')
        medicine_mfg_per_composition = browser.find_elements_by_xpath('//*[@id="productSearch"]/tbody/tr/td[2]')

        for i,j in zip(medicine_name_per_composition,medicine_mfg_per_composition):
            x=i.get_attribute("text")
            y=j.get_attribute('textContent')
            medicine_name.append(x)
            mfg_name.append(y)
            ids.append(_id)
    
    product_details['medicine_name']=medicine_name 
    product_details['mfg_name']=mfg_name
    product_details['comp_id']=ids
    print('done')
    browser.close()
    print(product_details)
    return product_details


In [17]:
cursor.execute("""create table medicine_list_(medicine_name varchar(200),mfg_name varchar(200),comp_id varchar(200))""")

ProgrammingError: 1050 (42S01): Table 'medicine_list_' already exists

In [21]:
cursor.execute("""select * from medicine_list_""")
for x in cursor:
    print(x)

('DECOMB 100MG TAB', 'ALDE MEDI IMPEX', '3089')
('DIPLOFEN 100MG TAB', 'PHARMATECH HEALTHCARE', '3089')
('DIPT TAB', 'RHINE BIOGENICS', '3089')
('DOLCAZ 100MG TAB', 'GLOVEL HEALTHCARE PVT LTD', '3089')
('DOLOKIND TAB', 'MANKIND PHARMA LTD', '3089')
('DOLOWIN TAB', 'MICRO LABS LIMITED', '3089')
('DOLOBEST TAB', 'TTK', '3089')
('DOLOSAID 100MG TAB', 'UNIVENTIS MEDICARE LTD', '3089')
('DORFEN 100MG TAB', 'TRIVENI FORMULATIONS', '3089')
('DUOFEN TAB', 'BEFAM PHARMACEUTICALS', '3089')
('ECLOFEN CAP', 'ABS REMEDIES', '3089')
('ECLONAC 100MG TAB', 'JAGSONPAL PHARMACEUTICALS PVT LTD', '3089')
('EDIFLAM TAB', 'EDEN HEALTHCARE', '3089')
('FAST FIX 100MG TAB', 'WELLMARK LIFE SCIENCES', '3089')
('FAZE 100MG TAB', 'ALBATROSS HEALTHCARE', '3089')
('FENAICA 100MG TAB', 'SPICA LAB', '3089')
('FENACLO 100MG TAB', 'BIOMAX BIOTECHNICS', '3089')
('FENBEST 100MG TAB', 'MANEESH PHARMACEUTICALS', '3089')
('FENOBAX 100MG TAB', 'BAXTON PHARMACIA', '3089')
('FENODOL TAB', 'WYNCLARK PHARMACEUTICALS PVT LTD', '30

('ETOGEN 120MG TAB', 'SALVEO LIFECARE', '5524')
('ETOGLO 120MG TAB', 'TYRAMED HEALTHCARE', '5524')
('ETOHIT 120MG TAB', 'HAUZ PHARMA', '5524')
('ETOICA 120MG TAB', 'SPICA LAB', '5524')
('ETOJET 120MG TAB', 'THEMIS MEDICARE LTD', '5524')
('ETOJEK 120MG TAB', 'ASTEMAX BIOTECH', '5524')
('ETOKINZ 120MG TAB', 'HENIN LUKING PHARMA PVT LTD', '5524')
('ETOLOL 120MG TAB', 'VIBCARE  PHARMA', '5524')
('ETOLEN 120MG TAB', 'ALENCURE BIOTECH PVT LTD', '5524')
('ETOMAV 120MG TAB', 'MAVERICK PHARMA', '5524')
('ETOM 120MG TAB', 'INTRA LABS INDIA PVT LTD', '5524')
('ETOMAC 120 TAB', 'MAC HEALTHCARE PVT LTD', '5524')
('ETONOW 120MG TAB', 'GRANDCURE HEALTHCARE PVT LTD', '5524')
('ETONEXT 120MG TAB', 'ZEE LABORATORIES LTD', '5524')
('ETONISE 120MG TAB', 'SOLITAIRE PHARMACIA PVT LTD', '5524')
('ETONEX 120MG TAB', 'NEXKEM PHARMACEUTIALS PVT LTD', '5524')
('ETONET 120MG TAB', 'LEZAA BIOTECH', '5524')
('ETOPRIME 120MG TAB', 'ALNA BIOTECH PVT LTD', '5524')
('ETORICA 120MG TAB', 'MICRO LABS LIMITED', '5524')
('

('NIMS CD TAB', 'SYMET LABS PVT LTD', '3269')
('NIMSY MD TAB', 'SERVOCARE LIFESCIENCES', '3269')
('NIMSIM 100MG TAB', 'ZENLABS ETHICA LTD', '3269')
('NIMSEL 100MG TAB', 'CUREX PHARMACEUTICALS PVT LTD', '3269')
('NIMTOP TAB', 'SCOTT EDIL PHARMACIA LTD', '3269')
('NIMTO 100MG TAB', 'RHOMBUS PHARMA PVT LTD', '3269')
('NIMTER MD TAB', 'AXTER PHARMACEUTICALS PVT LTD', '3269')
('NIMTEX 100MG TAB', 'TEXAS PHARMA', '3269')
('NIMULID 100MG TAB', 'PANACEA BIOTEC', '3269')
('NIMULID MD TAB', 'PANACEA BIOTEC', '3269')
('NIMULASE TAB', 'TTK', '3269')
('NIMUTAB TAB', 'CENTAUR PHARMACEUTICALS PVT LTD', '3269')
('NIMULOT TAB TAB', 'LOTUS ORGANS', '3269')
('NIMUPAIN 100MG TAB', 'CIPLA', '3269')
('NIMUWIN 100MG TAB', 'WINGS BIOTECH', '3269')
('NIMUGESIC AMBER 100MG TAB', 'ABBOTT', '3269')
('NIMUBID 100MG TAB', 'DENIZEN PHARMA', '3269')
('NIMUVISTA 100MG TAB', 'INNOVA FORMULATIONS PVT LTD', '3269')
('NIMUSYM 100MG TAB', 'SYMBIOSIS PHARMACEUTICAL PVT LTD', '3269')
('NIMUTAL 100MG TAB', 'SYMBIOSIS PHARMACE

('STP M TAB', 'PHARMATECH HEALTHCARE', '3710')
('STRAN MF TAB', 'OASIS BIOTECH', '3710')
('SYL TMF TAB', 'BENNET MYPHER PHARMA', '3710')
('SYNOSTAT MF TAB', 'SYNOKEM PHARMACEUTICALS LTD', '3710')
('FRAMIC M TAB', 'AXIAL PHARMACEUTICALS PVT LTD', '3710')
('FRANAX M TAB', 'FRANK MEDILINK', '3710')
('FUDART-MF TAB', 'INTRA LIFE', '3710')
('GEARIL MF TAB', 'WELLMARK LIFE SCIENCES', '3710')
('LAXAMIC M TAB', 'ZEE LABORATORIES LTD', '3710')
('ZOXISTAT TAB', 'MYRIADHUES HEALTHCARE', '3710')
('PASTATE MF TAB', 'PASTEUR PHARMA', '3710')
('PAUSE MF TAB', 'EMCURE PHARMACEUTICALS LTD', '3710')
('PAUSERA M TAB', 'PFIZER LIMITED', '3710')
('PAUSEVA TAB', 'VIVO LIFES SCIENCES PVT LTD', '3710')
('TAMIC MF TAB', 'TTK', '3710')
('TAM MF TAB', 'AMERICAN BIOCARE', '3710')
('TANAMIC M TAB', 'BAXTON PHARMACIA', '3710')
('TANCID MF TAB', 'GROWMAX MEDICARE PVT LTD', '3710')
('TANDEM M TAB', 'AAMORB PHARMACEUTICALS PVT LTD', '3710')
('TANDIN M TAB', 'PRIDE HEALTHCARE', '3710')
('TANEX MF TAB', 'CHEMO HEALTHCAR

('KOLO 650MG TAB', 'KANRAXY PHARMA', '5910')
('KOSPAR 650MG TAB', 'ELKOS HEALTHCARE', '5910')
('KRINMOL 650MG TAB', 'KRISHGIR PHARMA', '5910')
('KURIPARA 650MG TAB', 'RELIABLE PHARMA', '5910')
('KYOMOL 650MG TAB', 'LIFEKYOR PHARMACEUTICALS', '5910')
('K PAR 650MG TAB', 'KINESIS PHARMACEUTICALS PVT LTD', '5910')
('LAMOL 650MG TAB', 'DATLA LABORATORIES', '5910')
('LANOL ER 650MG TAB', 'HETERO HEALTHCARE LIMITED', '5910')
('LANTO P TAB', 'GLANTO PHARMA', '5910')
('LEEMOL 650MG TAB', 'LEADS PHARMA', '5910')
('LEKADOL 650MG TAB', 'ORGANIC LABORATORIES', '5910')
('LENAL 650MG TAB', 'HETERO HEALTHCARE LIMITED', '5910')
('LEO 650MG TAB', 'LEO STAR HEALTHCARE PVT LTD', '5910')
('GLAMOL 650MG TAB', 'GLADCARE FORMULATIONS PVT LTD', '5910')
('GLENPAR 650MG TAB', 'GLENMARK PHARMACEUTICALS LTD', '5910')
('GRACEMOL 650MG TAB', 'GRACE PHARMACEUTICALS', '5910')
('GRENO 650MG TAB', 'CHEMO HEALTHCARE PVT LTD', '5910')
('G MOL 650MG TAB', 'GOPAL LIFESCIENCES', '5910')
('FPMOL 650MG TAB', 'FINE BIO SCIENCE

...

('JIFON CV 200MG TAB', 'DR JOHNS LAB HEALTHCARE PVT LTD', '7556')
('JOHNCEF CL 200MG TAB', 'JOHNLEE PHARMACEUTICALS PVT LTD', '7556')
('JUNOFIX CV 325MG TAB', 'ZYNOVIA LIFECARE', '7556')
('KANCEF CV 325MG TAB', 'KANSAS LAB', '7556')
('KAVCEF CL 200MG TAB', 'KAVYA HEALTHCARE', '7556')
('KEFCLAV 200MG TAB', 'MACLEODS PHARMACEUTICALS LTD', '7556')
('KEFEX CV 325MG TAB', 'AZILLIAN HEALTHCARE', '7556')
('KEMFIX CV 200MG TAB', 'ANIKEM LABORATORIES', '7556')
('KEYET CV TAB', 'XIEON LIFE SCIENCES PVT LTD', '7556')
('ORFIX CV TAB', 'ORCHID', '7556')
('ORIFIX CV 200MG TAB', 'OSCAR REMEDIES PVT LTD', '7556')
('OSIXIM CV 325MG TAB', 'ORLIFE HEALTHCARE', '7556')
('OURXIME CV 325MG TAB', 'MEDIVAXIA PHARMA', '7556')
('OVACEF CV 325MG TAB', 'ROYAL DISTRIBUTION CO PVT LTD', '7556')
('PHAGOCEF CV 325MG TAB', 'HEALTHKIND LABS', '7556')
('PINCEF CV 200MG TAB', 'BRD MEDILABS', '7556')
('PINFIX CV 200MG TAB', 'PINARC LIFE SCIENCES', '7556')
('PODOCEF CV 325MG TAB', 'BIOCHEM PHARMACEUTICAL INDUSTRIES', '7556

('MOXPIC CA 325MG TAB', 'ALPIC BIOTECH LTD', '5488')
('MOXPIN CV 375MG TAB', 'PINARC LIFE SCIENCES', '5488')
('MOXSURE CV 375MG TAB', 'VINTAS PHARMA', '5488')
('MOXSPEN CL 375MG TAB', 'MADHAV BIOTECH', '5488')
('MOXTAS CV 375MG TAB', 'WINGS PHARMA', '5488')
('MOXVIA CV 375MG TAB', 'ELIVIA LIFE SCIENCES', '5488')
('MOXXIL DUO 375MG TAB', 'GUJARAT TERCE LABORATORIES', '5488')
('MOXYCLAV 375MG TAB', 'SYSTEMIC HEALTHCARE', '5488')
('MOXY PLUS CV 375MG TAB', 'YACCA PHARMACEUTICALS', '5488')
('MOXY CV 375MG TAB', 'JANUS REMEDIES', '5488')
('MOXYCON 375MG TAB', 'MAGMA ALLIANZ LABS LTD', '5488')
('MOXYNATE CV 375MG TAB', 'SHATAYUSHI HEALTH CARE', '5488')
('MOXYCAD CV 375MG TAB', 'CADWELL PHARMA PVT LTD', '5488')
('MOXY CL 375MG TAB', 'SARAL LABORATORIES', '5488')
('MOX CV 375MG TAB', 'SUN PHARMACEUTICAL INDUSTRIES LTD', '5488')
('MPOX CV 375MG TAB', 'LYKA HETERO', '5488')
('MPX CV 375MG TAB', 'LYKA HETERO', '5488')
('MQ CLAV 375MG TAB', 'MEDIQUEST INC', '5488')
('MUCOCLAV 375MG TAB', 'ORDAIN H

('HISERT 8MG TAB', 'ESMATRIX LIFESCIENCES PVT LTD', '7567')
('HISTAVERT 8MG TAB', 'PULSE PHARMACEUTICALS PVT LTD', '7567')
('HISTIGO 8MG TAB', 'MOLEKULE', '7567')
('HISTANIL 8MG TAB', 'SIDEL', '7567')
('HISTIAGO 8MG TAB', 'ACCILEX NUTRICORP', '7567')
('HISTUS 8MG TAB', 'TUSKER PHARMA PVT LTD', '7567')
('HISTABUS 8MG TAB', 'GLOBUS REMEDIES LTD', '7567')
('HIST 8MG TAB', 'SWIZINGER HEALTH CARE PVT LTD', '7567')
('HISTIMET 8MG TAB', 'METLAR FORMULATIONS', '7567')
('HISTWEL 8MG TAB', 'SOLITAIRE PHARMACIA PVT LTD', '7567')
('HISTABET 8MG TAB', 'TRIUMPH PHARMA', '7567')
('HISTY 8MG TAB', 'SOLITAIRE PHARMACIA PVT LTD', '7567')
('HISTATOP 8MG TAB', 'SOLITAIRE PHARMACIA PVT LTD', '7567')
('HISTUF 8MG TAB', 'AMERICAN BIOCARE', '7567')
('HISTASEA 8MG TAB', 'AMERICAN BIOCARE', '7567')
('HISTWELL 8MG TAB', 'TRIPADA HEALTHCARE PVT LTD', '7567')
('HISTICARE 8MG TAB', 'KRISH CARE REMEDYS PVT LTD', '7567')
('HISVERT 8MG TAB', 'HAEAL PHARMA', '7567')
('HITIN 8MG TAB', 'SBS BIOTECH', '7567')
('HYBET 8MG 

('FLUGRAINE 5MG TAB', 'MANO', '7180')
('FLUGRAIN 5MG TAB', 'RELIANCE FORMULATION PVT LTD', '7180')
('FLULINE 5MG TAB', 'JOHNLEE PHARMACEUTICALS PVT LTD', '7180')
('FLUMAC 5MG TAB', 'PRIDE HEALTHCARE', '7180')
('FLUMIG 5MG TAB', 'TALENT HEALTHCARE', '7180')
('FLUNARIN 5MG TAB', 'FDC LIMITED', '7180')
('FLUNER 5MG TAB', 'GENO PHARMA', '7180')
('FLUNAREST 5MG TAB', 'SENSES PHARMACEUTICALS LTD', '7180')
('FLUNA 5MG TAB', 'ASWINS PHARMACEUTICALS', '7180')
('FLUNANEO 5MG TAB', 'NEOCARDIAB CARE', '7180')
('FLUNIPAR 5MG TAB', 'EMENOX HEALTHCARE', '7180')
('FLUNAREX 5MG TAB', 'AUSTRAK PVT LTD', '7180')
('FLUNATOD 5MG TAB', 'ENTOD PHARMACEUTICALS LTD', '7180')
('FLUNARGEN 5MG TAB', 'GENOME THERAPEUTICS', '7180')
('FLUNEX 5MG TAB', 'THEO PHARMA PVT LTD', '7180')
('FLUNALINK 5MG TAB', 'ZOTA HEALTH CARE LTD', '7180')
('FLUNAHIST 5MG TAB', 'IKON REMEDIES PVT LTD', '7180')
('FLUNAXYL 5MG TAB', 'INVISION MEDI SCIENCE PVT LTD', '7180')
('FLUNADIN 5MG TAB', 'INNOVATIVE PHARMA', '7180')
('FLUNAWYN 5MG TA

('CITAQUIL FORTE TAB', 'TORNADO PHARMA', '7891')
('CITALOMAX TAB', 'METLAR FORMULATIONS', '7891')
('CITAFORD S PLUS 10MG TAB', 'OXFORD PHARMA', '7891')
('CITONIN PLUS TAB', 'UNIVENTIS MEDICARE LTD', '7891')
('CITOLEX PLUS TAB', 'LIFESAVERS REMEDIES', '7891')
('CITOGAL PLUS 10MG TAB', 'REGAL HEALTHCARE', '7891')
('CLAM PLUS TAB', 'SAINT PHARMA', '7891')
('CLEPS ES 0.5MG TAB', 'ZEE LABORATORIES LTD', '7891')
('CLODEP PLUS 0.5MG TAB', 'PARASOL LABORATORIES', '7891')
('CLONOTRIL PLUS TAB', 'TORRENT PHARMACEUTICALS LTD', '7891')
('CLONAFIT PLUS TAB', 'MANKIND PHARMA LTD', '7891')
('CLONETIC ES TAB', 'TRIPADA LIFECARE PVT LTD', '7891')
('CLONOGEM PLUS TAB', 'RONYD HEALTHCARE', '7891')
('CLONAXYL PLUS TAB', 'MARC LABORATORIES PVT LTD', '7891')
('CLONATEX PLUS TAB', 'ZANS PHARMA', '7891')
('CLONANZA PLUS TAB', 'MAS HEALTHCARE PVT LTD', '7891')
('CLONAGE S 10MG TAB', 'ALLENGE INDIA', '7891')
('CLONOZ PLUS TAB', 'SYCAP PHARMA', '7891')
('CLONAZ FORTE TAB', 'DIAL PHARMACEUTICALS ', '7891')
('CLON

In [20]:
df=pd.read_csv('medicines.csv')
cols = "`,`".join([str(i) for i in df.columns.tolist()])
for i,row in df.iterrows():
    sql = "INSERT INTO `medicine_list_` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))
    conn.commit()
    

In [28]:
print(df)

            medicine_name                   mfg_name comp_id
0        DECOMB 100MG TAB            ALDE MEDI IMPEX    3089
1      DIPLOFEN 100MG TAB      PHARMATECH HEALTHCARE    3089
2                DIPT TAB            RHINE BIOGENICS    3089
3        DOLCAZ 100MG TAB  GLOVEL HEALTHCARE PVT LTD    3089
4            DOLOKIND TAB         MANKIND PHARMA LTD    3089
...                   ...                        ...     ...
44509       ZOMARK FX TAB  UNIMARCK PHARMA INDIA LTD   24357
44510   VEXITINE PLUS TAB     VISION BIOLIFE PVT LTD   24357
44511          ZAM AD TAB             BLUBELL PHARMA   24357
44512         ZEDEP A TAB      TAS MED INDIA PVT LTD   24357
44513      ZEXTO PLUS TAB      TRIKO PHARMACEUTICALS   24357

[44514 rows x 3 columns]


In [29]:
df[['medicine_name','mfg_name','comp_id']].to_csv('medicines.csv',index=False)